# Average EHR Spending by City

In [135]:
## Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
import seaborn as sns

In [136]:
## Load the datasets
df_ehr_provider = pd.read_csv('EHR_Incentive_Program_Payments_Providers.csv')
df_ehr_hospital = pd.read_csv('EHR_Incentive_Program_Payments_Hospitals.csv')

In [137]:
## delete the unneeded columns on the provider dataset
df_ehr_provider.drop(df_ehr_provider.columns[0:8],axis=1,inplace = True)
df_ehr_provider.drop(df_ehr_provider.columns[1:4],axis=1,inplace = True)
df_ehr_provider.drop(df_ehr_provider.columns[3:6],axis=1,inplace = True)
df_ehr_provider.drop(['Program_Ye','Payee_NPI','Latitude','Longitude','total_rece'],axis=1,inplace = True)
df_ehr_provider.head()


,Business_C,Payment_Ye,total_paym
0,Sacramento,2019,38250.0
1,Oakland,2014,21250.0
2,Antioch,2016,21250.0
3,Loma Linda,2015,29750.0
4,Orange,2014,21250.0


In [138]:
## group the practitioner data by city and year, then sum the total payments
df_ehr_provider['Business_C'] = df_ehr_provider['Business_C'].str.lower()
df_ehr_provider = df_ehr_provider.groupby(['Business_C','Payment_Ye'])['total_paym'].sum().reset_index()
df_ehr_provider.to_csv('test_group_by_check.csv')
df_ehr_provider.head()

,Business_C,Payment_Ye,total_paym
0,adelanto,2014,14166.67
1,adelanto,2015,21250.00
2,adelanto,2016,29750.00
3,agoura hills,2014,63750.00
4,agoura hills,2016,21250.00


In [139]:
#change all Entity Names to lower case
df_city_revenue['Entity Name'] = df_city_revenue['Entity Name'].str.lower()
df_city_revenue.head()

,Entity Name,Fiscal Year,Total Revenues,Estimated Population,Revenues Per Capita
0,adelanto,2021,22556106,36569.0,617.0
1,adelanto,2020,20455012,35663.0,574.0
2,adelanto,2019,20232000,35136.0,576.0
3,adelanto,2018,18224861,35293.0,516.0
4,adelanto,2017,15171753,34273.0,443.0


In [140]:
df_provider_city = df_ehr_provider.merge(df_city_revenue, how="inner",left_on=['Business_C','Payment_Ye'],right_on=['Entity Name','Fiscal Year'])
df_provider_city.head()

,Business_C,Payment_Ye,total_paym,Entity Name,Fiscal Year,Total Revenues,Estimated Population,Revenues Per Capita
0,adelanto,2014,14166.67,adelanto,2014,13047272,32511.0,401.0
1,adelanto,2015,21250.00,adelanto,2015,14051611,33084.0,425.0
2,adelanto,2016,29750.00,adelanto,2016,14565863,33497.0,435.0
3,agoura hills,2014,63750.00,agoura hills,2014,25485389,20625.0,1236.0
4,agoura hills,2016,21250.00,agoura hills,2016,28100766,21211.0,1325.0


In [141]:
df_provider_city = df_provider_city.drop(['Entity Name','Fiscal Year'], axis=1)
df_provider_city.head()

,Business_C,Payment_Ye,total_paym,Total Revenues,Estimated Population,Revenues Per Capita
0,adelanto,2014,14166.67,13047272,32511.0,401.0
1,adelanto,2015,21250.00,14051611,33084.0,425.0
2,adelanto,2016,29750.00,14565863,33497.0,435.0
3,agoura hills,2014,63750.00,25485389,20625.0,1236.0
4,agoura hills,2016,21250.00,28100766,21211.0,1325.0


In [143]:
df_provider_city.to_csv('ultimate_test.csv')

In [ ]:
df_provider_city.head()